# TextAttack End-to-End

This tutorial provides a broad end-to-end overview of training, evaluating, and attacking a model using TextAttack.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QData/TextAttack/blob/master/docs/2notebook/0_End_to_End.ipynb)

[![View Source on GitHub](https://img.shields.io/badge/github-view%20source-black.svg)](https://github.com/QData/TextAttack/blob/master/docs/2notebook/0_End_to_End.ipynb)

## Training

First, we're going to train a model. TextAttack integrates directly with [transformers](https://github.com/huggingface/transformers/) and [datasets](https://github.com/huggingface/datasets) to train any of the `transformers` pre-trained models on datasets from `datasets`. 

Let's use the Rotten Tomatoes Movie Review dataset: it's relatively short , and showcasesthe key features of `textattack train`. Let's take a look at the dataset using `textattack peek-dataset`:

In [1]:
!textattack peek-dataset --dataset-from-huggingface rotten_tomatoes

Using custom data configuration default
Reusing dataset rotten_tomatoes_movie_review (/home/srujanjoshi/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/e06abb624abab47e1a64608fdfe65a913f5a68c66118408032644a3285208fb5)
textattack: Loading datasets dataset rotten_tomatoes, split train.
textattack: Number of samples: 8530
textattack: Number of words per input:
textattack: 	total:   157755
textattack: 	mean:    18.49
textattack: 	std:     8.58
textattack: 	min:     1
textattack: 	max:     51
textattack: Dataset lowercased: True
textattack: First sample:
the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal . 

textattack: Last sample:
things really get weird , though not particularly scary : the movie is all portent and no content . 

textattack: Found 2 distinct outputs.
textattack: Most common outputs:
	 1      (4265)
	 0      (4265)


The dataset looks good! It's lowercased already, so we'll make sure our model is uncased. The longest input is 51 words, so we can cap our maximum sequence length (`--model-max-length`) at 64.

We'll train [`distilbert-base-uncased`](https://huggingface.co/transformers/model_doc/distilbert.html), since it's a relatively small model, and a good example of how we integrate with `transformers`.

So we have our command:

```bash
textattack train                      \ # Train a model with TextAttack
    --model distilbert-base-uncased   \ # Using distilbert, uncased version, from `transformers`
    --dataset rotten_tomatoes         \ # On the Rotten Tomatoes dataset
    --model-num-labels 3              \ # That has 2 labels
    --model-max-length 64             \ # With a maximum sequence length of 64
    --per-device-train-batch-size 128 \ # And batch size of 128
    --num-epochs 3                    \ # For 3 epochs 
```

Now let's run it:

In [2]:
!textattack train --model-name-or-path distilbert-base-uncased --dataset rotten_tomatoes --model-num-labels 2 --model-max-length 64 --per-device-train-batch-size 128 --num-epochs 3

textattack: Loading transformers AutoModelForSequenceClassification: distilbert-base-uncased
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-bas

## Evaluation

We successfully fine-tuned `distilbert-base-cased` for 3 epochs. Now let's evaluate it using `textattack eval`. This is as simple as providing the path to the pretrained model to `--model`, along with the number of evaluation samples. `textattack eval` will automatically load the evaluation data from training:

In [4]:
!textattack eval --num-examples 1000 --model ./outputs/2021-10-08-01-32-25-711356/best_model/ --dataset-from-huggingface rotten_tomatoes --dataset-split test

Using custom data configuration default
Reusing dataset rotten_tomatoes_movie_review (/home/srujanjoshi/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/e06abb624abab47e1a64608fdfe65a913f5a68c66118408032644a3285208fb5)
textattack: Loading datasets dataset rotten_tomatoes, split test.
textattack: Got 1000 predictions.
textattack: Correct 849/1000 (84.90%)


Awesome -- we were able to train a model up to 84.9% accuracy on the test dataset – with only a single command!

## Attack

Finally, let's attack our pre-trained model. We can do this the same way as before (by providing the path to the pretrained model to `--model`). For our attack, let's use the "TextFooler" attack recipe, from the paper ["Is BERT Really Robust? A Strong Baseline for Natural Language Attack on Text Classification and Entailment" (Jin et al, 2019)](https://arxiv.org/abs/1907.11932). We can do this by passing `--recipe textfooler` to `textattack attack`.

> *Warning*: We're printing out 1000 examples and, if the attack succeeds, their perturbations. The output of this command is going to be quite long!


In [7]:
!textattack attack --recipe textfooler --num-examples 1000 --model ./outputs/2021-10-08-01-32-25-711356/best_model/ --dataset-from-huggingface rotten_tomatoes --dataset-split test

Using custom data configuration default
Reusing dataset rotten_tomatoes_movie_review (/home/srujanjoshi/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/e06abb624abab47e1a64608fdfe65a913f5a68c66118408032644a3285208fb5)
textattack: Loading datasets dataset rotten_tomatoes, split test.
textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_

[Succeeded / Failed / Skipped / Total] 5 / 0 / 4 / 9:   1%| | 10/1000 [00:04<07:--------------------------------------------- Result 10 ---------------------------------------------
Positive (98%) --> Negative (90%)

generates an enormous feeling of empathy for its characters .

generates an dreaded feeling of empathy for its characters .


[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10:   1%| | 10/1000 [00:04<07--------------------------------------------- Result 11 ---------------------------------------------
Positive (98%) --> Negative (89%)

exposing the ways we fool ourselves is one hour photo's real strength .

exposing the routing we fool ourselves is one hour photo's real strenght .


[Succeeded / Failed / Skipped / Total] 7 / 0 / 4 / 11:   1%| | 11/1000 [00:04<06--------------------------------------------- Result 12 ---------------------------------------------
Positive (53%) --> Negative (50%)

it's up to you to decide whether to admire these people's dedication to t

[Succeeded / Failed / Skipped / Total] 19 / 0 / 7 / 26:   3%| | 26/1000 [00:06<0--------------------------------------------- Result 27 ---------------------------------------------
Positive (93%) --> Negative (90%)

. . . a good film that must have baffled the folks in the marketing department .

. . . a ok film that must have baffled the folks in the marketing department .


[Succeeded / Failed / Skipped / Total] 20 / 0 / 7 / 27:   3%| | 28/1000 [00:07<0--------------------------------------------- Result 28 ---------------------------------------------
Positive (98%) --> Negative (56%)

. . . is funny in the way that makes you ache with sadness ( the way chekhov is funny ) , profound without ever being self-important , warm without ever succumbing to sentimentality .

. . . is outlandish in the trajectories that makes you ache with sadness ( the way chekhov is funny ) , shum without ever being self-important , warm without ever fending to sentimentality .


[Succeeded / Failed / Ski

[Succeeded / Failed / Skipped / Total] 32 / 1 / 12 / 45:   5%| | 46/1000 [00:08<--------------------------------------------- Result 46 ---------------------------------------------
Positive (85%) --> Negative (81%)

goldmember is funny enough to justify the embarrassment of bringing a barf bag to the moviehouse .

goldmember is comical enough to justify the embarrassment of bringing a barf bag to the moviehouse .


[Succeeded / Failed / Skipped / Total] 33 / 1 / 12 / 46:   5%| | 46/1000 [00:08<--------------------------------------------- Result 47 ---------------------------------------------
Positive (78%) --> Negative (94%)

. . . a fairly disposable yet still entertaining b picture .

. . . a fairly disposable yet still droll b picture .


[Succeeded / Failed / Skipped / Total] 34 / 1 / 12 / 47:   5%| | 47/1000 [00:08<--------------------------------------------- Result 48 ---------------------------------------------
Positive (98%) --> Negative (52%)

it may not be particularly i

[Succeeded / Failed / Skipped / Total] 46 / 1 / 15 / 62:   6%| | 62/1000 [00:10<--------------------------------------------- Result 63 ---------------------------------------------
Positive (98%) --> Negative (55%)

this is one of those rare docs that paints a grand picture of an era and makes the journey feel like a party .

this is one of those rare docs that paint a gargantuan picture of an timeframe and ai the jaunt feel like a party .


[Succeeded / Failed / Skipped / Total] 47 / 1 / 15 / 63:   6%| | 64/1000 [00:10<--------------------------------------------- Result 64 ---------------------------------------------
Positive (98%) --> Negative (94%)

poignant if familiar story of a young person suspended between two cultures .

dreaded if familiar story of a young person suspended between two cultures .


[Succeeded / Failed / Skipped / Total] 48 / 1 / 15 / 64:   6%| | 64/1000 [00:10<--------------------------------------------- Result 65 ------------------------------------------

...

[Succeeded / Failed / Skipped / Total] 741 / 36 / 137 / 914:  91%|▉| 914/1000 [0--------------------------------------------- Result 915 ---------------------------------------------
Negative (95%) --> Positive (79%)

the movie tries to be ethereal , but ends up seeming goofy .

the movie tries to be ethereal , but fin up seeming fun .


[Succeeded / Failed / Skipped / Total] 742 / 36 / 137 / 915:  92%|▉| 916/1000 [0--------------------------------------------- Result 916 ---------------------------------------------
Negative (94%) --> [FAILED]

i was hoping that it would be sleazy and fun , but it was neither .


[Succeeded / Failed / Skipped / Total] 742 / 37 / 137 / 916:  92%|▉| 916/1000 [0--------------------------------------------- Result 917 ---------------------------------------------
Negative (97%) --> Positive (69%)

harris is supposed to be the star of the story , but comes across as pretty dull and wooden .

harris is supposed to be the star of the story , but comes across

[Succeeded / Failed / Skipped / Total] 753 / 39 / 141 / 933:  93%|▉| 933/1000 [0--------------------------------------------- Result 934 ---------------------------------------------
Negative (97%) --> Positive (57%)

the premise itself is just sooooo tired . pair that with really poor comedic writing . . . and you've got a huge mess .

the premise itself is just sooooo exhausted . pair that with really tenuous comedic writing . . . and you've got a huge mayhem .


[Succeeded / Failed / Skipped / Total] 754 / 39 / 141 / 934:  93%|▉| 934/1000 [0--------------------------------------------- Result 935 ---------------------------------------------
Positive (94%) --> [SKIPPED]

proves a lovely trifle that , unfortunately , is a little too in love with its own cuteness .


[Succeeded / Failed / Skipped / Total] 754 / 39 / 142 / 935:  94%|▉| 936/1000 [0--------------------------------------------- Result 936 ---------------------------------------------
Negative (86%) --> Positive (84%)

did

[Succeeded / Failed / Skipped / Total] 765 / 41 / 145 / 951:  95%|▉| 951/1000 [0--------------------------------------------- Result 952 ---------------------------------------------
Negative (90%) --> Positive (97%)

essentially , the film is weak on detail and strong on personality

essentially , the film is delicate on detail and strong on personality


[Succeeded / Failed / Skipped / Total] 766 / 41 / 145 / 952:  95%|▉| 952/1000 [0--------------------------------------------- Result 953 ---------------------------------------------
Negative (88%) --> Positive (52%)

a relentless , bombastic and ultimately empty world war ii action flick .

a relentless , bombastic and ultimately empty monde war ii action gesture .


[Succeeded / Failed / Skipped / Total] 767 / 41 / 145 / 953:  95%|▉| 953/1000 [0--------------------------------------------- Result 954 ---------------------------------------------
Negative (95%) --> Positive (55%)

[hell is] looking down at your watch and realizing s

[Succeeded / Failed / Skipped / Total] 781 / 42 / 146 / 969:  97%|▉| 969/1000 [0--------------------------------------------- Result 970 ---------------------------------------------
Negative (93%) --> Positive (64%)

'dragonfly' dwells on crossing-over mumbo jumbo , manipulative sentimentality , and sappy dialogue .

'dragonfly' dwells on crossing-over mumbo jumbo , manipulative sentimentality , and viscous dialogue .


[Succeeded / Failed / Skipped / Total] 782 / 42 / 146 / 970:  97%|▉| 971/1000 [0--------------------------------------------- Result 971 ---------------------------------------------
Negative (96%) --> Positive (66%)

in his determination to lighten the heavy subject matter , silberling also , to a certain extent , trivializes the movie with too many nervous gags and pratfalls .

in his defining to sweeten the intense subject facet , silberling also , to a certain earnestness , trivializes the filmmakers with too disparate nervous farce and pratfalls .


[Succeeded / F

[Succeeded / Failed / Skipped / Total] 797 / 42 / 146 / 985:  99%|▉| 986/1000 [0--------------------------------------------- Result 986 ---------------------------------------------
Negative (96%) --> Positive (68%)

debut effort by " project greenlight " winner is sappy and amateurish .

debut effort by " project greenlight " winner is viscous and amateur .


[Succeeded / Failed / Skipped / Total] 798 / 42 / 146 / 986:  99%|▉| 986/1000 [0--------------------------------------------- Result 987 ---------------------------------------------
Negative (95%) --> Positive (65%)

one gets the impression the creators of don't ask don't tell laughed a hell of a lot at their own jokes . too bad none of it is funny .

one is the impression the creators of don't ask don't tell laughed a hell of a lot at their unique jokes . too bad none of it is offbeat .


[Succeeded / Failed / Skipped / Total] 799 / 42 / 146 / 987:  99%|▉| 987/1000 [0--------------------------------------------- Result 988 ---

Looks like our model was 84.9% successful (makes sense - same evaluation set as `textattack eval`!), meaning that TextAttack attacked the model with 849 examples (since the attack won't run if an example is originally mispredicted). The attack success rate was 94.82%, meaning that TextFooler failed to find an adversarial example only 6.47% of the time.


## Conclusion

That's all, folks! We've learned how to train, evaluate, and attack a model with TextAttack, using only three commands! 😀